In [ ]:
using ReactiveMP, Distributions, Random, Rocket, StableRNGs

In [ ]:
f(x) = abs(exp(x) * sin(x))
rng = StableRNG(123)

@model function nonlinear_estimation(n)
    
    θ ~ NormalMeanVariance(0.0, 100.0)
    m ~ NormalMeanVariance(0.0, 1.0)
    
    # SamplingApproximation(nsamples, generator, seed)
    w ~ f(θ) where { meta = SamplingApproximation(1000, rng) }
    
    y = datavar(Float64, n)
    
    for i in 1:n
        y[i] ~ NormalMeanPrecision(m, w)
    end
    
    return θ, m, w, y
end

In [ ]:
@constraints function nconstsraints(nsamples)
    q(θ) :: SampleList(nsamples, LeftProposal())
    q(w) :: SampleList(nsamples, RightProposal())
    q(θ, w, m) = q(θ)q(m)q(w)
end

In [ ]:
seed = 123
rng  = MersenneTwister(seed)

niters   = 15 # Number of VMP iterations
nsamples = 5_000 # Number of samples in approximation

n = 500 # Number of IID samples
μ = -10.0
θ = -1.0
w = f(θ)

data = rand(rng, NormalMeanPrecision(μ, w), n);

In [ ]:
result = inference(
    model = Model(nonlinear_estimation, n),
    constraints = nconstsraints(nsamples),
    data = (y = data, ), 
    initmarginals = (m = vague(NormalMeanPrecision), w = vague(Gamma)),
    returnvars = (θ = KeepLast(), ),
    iterations = niters,  
    showprogress = true
)